In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.metrics import MeanIoU



In [2]:
image_dir=r'dataaset\images'
mask_dir=r'dataaset\masks'

image_size = (224, 224)
num_classes = 3
batch_size=2
epochs=10

image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])

In [3]:
image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])

assert len(image_paths) == len(mask_paths), "Mismatch in number of images and masks"
print("Total pairs:", len(image_paths))

Total pairs: 109


In [4]:
def load_and_preprocess(image_path, mask_path):
    # Load and resize image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32) / 255.0

    # Load and resize mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Force 1 channel
    mask = tf.image.resize(mask,image_size, method='nearest')
    mask = tf.squeeze(mask, axis=-1)              # Remove channel
    mask = tf.cast(mask, tf.int32)                # For sparse_categorical_crossentropy

    return image, mask


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.cache().shuffle(buffer_size=100)
train_ds = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [6]:
for batch_images, batch_masks in train_ds.take(1):
    print(" Image batch shape:", batch_images.shape)
    print(" Mask batch shape:", batch_masks.shape)

 Image batch shape: (2, 224, 224, 3)
 Mask batch shape: (2, 224, 224)


In [7]:
def upsample_block(filters,kernel_size):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2DTranspose(filters,kernel_size,strides=2,padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU()
    ])

In [8]:
def build_unet_mobilenetv2(input_shape, num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    # Use intermediate layers for skip connections
    layer_names = [
        'block_1_expand_relu',   # 112x112
        'block_3_expand_relu',   # 56x56
        'block_6_expand_relu',   # 28x28
        'block_13_expand_relu',  # 14x14
        'out_relu'               # 7x7
    ]
    layers = [base_model.get_layer(name).output for name in layer_names]

    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
    down_stack.trainable = True  # Enable fine-tuning

    inputs = tf.keras.Input(shape=input_shape)
    skips = down_stack(inputs)
    x = skips[-1]  # Start from 'out_relu' (7x7)

    # Decoder: upsampling and concatenation with skip connections
    x = upsample_block(512, 3)(x)           # 14x14
    x = tf.keras.layers.Concatenate()([x, skips[-2]])

    x = upsample_block(256, 3)(x)           # 28x28
    x = tf.keras.layers.Concatenate()([x, skips[-3]])

    x = upsample_block(128, 3)(x)           # 56x56
    x = tf.keras.layers.Concatenate()([x, skips[-4]])

    x = upsample_block(64, 3)(x)            # 112x112
    x = tf.keras.layers.Concatenate()([x, skips[-5]])

    x = upsample_block(32, 3)(x)            # 224x224

    # Final segmentation head
    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

In [10]:
model = build_unet_mobilenetv2(image_size + (3,), num_classes)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ [(None, 112, 112, │  2,257,984 │ input_layer_1[0]… │
│ (Functional)        │ 96), (None, 56,   │            │                   │
│                     │ 56, 144), (None,  │            │                   │
│                     │ 28, 28, 192),     │            │                   │
│                     │ (None, 14, 14,    │            │                   │
│                     │ 576), (None, 7,   │            │                   │
│                     │ 7, 1280)]         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 14, 14,    │  5,900,800 │ functional[0][4]  │
│ (Sequential)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 14, 14,    │          0 │ sequential[0][0], │
│ (Concatenate)       │ 1088)             │            │ functional[0][3]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 28, 28,    │  2,508,032 │ concatenate[0][0] │
│ (Sequential)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 28, 28,    │          0 │ sequential_1[0][… │
│ (Concatenate)       │ 448)              │            │ functional[0][2]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 56, 56,    │    516,736 │ concatenate_1[0]… │
│ (Sequential)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 56, 56,    │          0 │ sequential_2[0][… │
│ (Concatenate)       │ 272)              │            │ functional[0][1]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 112, 112,  │    156,992 │ concatenate_2[0]… │
│ (Sequential)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 112, 112,  │          0 │ sequential_3[0][… │
│ (Concatenate)       │ 160)              │            │ functional[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 224, 224,  │     46,240 │ concatenate_3[0]… │
│ (Sequential)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │         99 │ sequential_4[0][… │
│                     │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,386,883 (43.44 MB)

 Trainable params: 11,350,787 (43.30 MB)

 Non-trainable params: 36,096 (141.00 KB)

In [11]:
history = model.fit(train_ds, epochs=epochs,callbacks=[early_stop])


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 93s 955ms/step - accuracy: 0.6632 - loss: 0.7352
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 49s 895ms/step - accuracy: 0.8221 - loss: 0.3864
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 49s 896ms/step - accuracy: 0.8375 - loss: 0.3978
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 53s 958ms/step - accuracy: 0.8591 - loss: 0.3287
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 49s 881ms/step - accuracy: 0.8634 - loss: 0.3197
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 51s 932ms/step - accuracy: 0.8855 - loss: 0.2660
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 51s 931ms/step - accuracy: 0.8943 - loss: 0.2520
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 51s 933ms/step - accuracy: 0.8959 - loss: 0.2626
Epoch 9/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 48s 874ms/step - accuracy: 0.9011 - loss: 0.2491
Epoch 10/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 52s 953ms/step - accuracy: 0.9016 - loss: 0.2345


In [12]:
model.save('unet_mobilenetv2.h5')